<a href="https://colab.research.google.com/github/erin-carroll14/AspenRS/blob/main/dist_RF_RMBL_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import earth engine library
import ee
import folium
from folium import plugins

# establish connection to own earth engine account
ee.Authenticate()
ee.Initialize()

In [ ]:
# call 2020 imagery stack
!git clone https://www.github.com/erin-carroll14/AspenRS.git

%cd /content/AspenRS

from S2SRstack2020 import stacked2020_JunOct, Colorado

In [ ]:
# Veg mask
col = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(Colorado).filterDate('2020-01-01', '2021-01-01')

# function to get only pixels classified as veg at any point in year
def getVeg(img):
  return img.mask(img.select('SCL').eq(4))

colVeg = col.map(getVeg)
imgVeg = colVeg.qualityMosaic('SCL')

# mask stacked imagery by 2020 veg mask
stackVeg = stacked2020_JunOct.mask(imgVeg.select('SCL'))


In [ ]:
# clip to RMBL
RMBL = ee.Geometry.Polygon([[-107.22694538043767,38.75434750566651],
                            [-106.74629352496892,38.75434750566651],
                            [-106.74629352496892,39.08236920480586],
                            [-107.22694538043767,39.08236920480586],
                            [-107.22694538043767,38.75434750566651]])

stackVegRMBL2020 = stackVeg.clip(RMBL)

In [ ]:
# prepare reference data

# load aspen reference points
aspen = ee.FeatureCollection('users/erin_carroll/aspenAssets/refAspen')

# add 'class' field (1=aspen presence)
def getClass1(f):
  return f.set('class', 1)
aspen = aspen.map(getClass1)

# load non-aspen reference points
nonAspen = ee.FeatureCollection('users/erin_carroll/aspenAssets/refNonAspen')

# add 'class' field (0=absence)
def getClass0(f):
  return f.set('class', 0)
nonAspen = nonAspen.map(getClass0)

# merge aspen and non-aspen points
ref = aspen.merge(nonAspen)

In [ ]:
# prepare training data from reference data

# extract image bands at reference points, keep 'class' property from reference points
reference = stackVegRMBL2020.sampleRegions(**{
  'collection': ref,
  'properties': ['class'],
  'scale': 10
})

# subset validation sample (25%)
random = reference.randomColumn('x')
validation = random.filter(ee.Filter.lt('x', 0.25))
training = random.filter(ee.Filter.gte('x', 0.25))


In [ ]:
# create and train model
bandsAll = ['B2', 'B2_1', 'B2_2', 'B2_3', 'B2_4', 'B3', 'B3_1', 'B3_2', 'B3_3', 'B3_4', 'B4', 'B4_1', 'B4_2', 'B4_3', 'B4_4', 'B5', 'B5_1', 'B5_2', 'B5_3', 'B5_4', 'B6', 'B6_1', 'B6_2', 'B6_3', 'B6_4', 'B7', 'B7_1', 'B7_2', 'B7_3', 'B7_4', 'B8', 'B8_1', 'B8_2', 'B8_3', 'B8_4', 'B8A', 'B8A_1', 'B8A_2', 'B8A_3', 'B8A_4', 'B11', 'B11_1', 'B11_2', 'B11_3','B11_4', 'B12', 'B12_1', 'B12_2', 'B12_3', 'B12_4']

classifier = ee.Classifier.smileRandomForest(10).train(**{
      'features': training,
      'classProperty': 'class',
      'inputProperties': bandsAll
    });

In [ ]:
# deploy model
aspenMask2020 = stackVegRMBL2020.classify(classifier)

In [ ]:
# confusion matrices

# training
trainAccuracy = classifier.confusionMatrix()
print('Training error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

# validation
validated = validation.classify(classifier)
valAccuracy = validated.errorMatrix('class', 'classification')
print('Validation error matrix: ', valAccuracy.getInfo())
print('Validation overall accuracy: ', valAccuracy.accuracy().getInfo())

In [ ]:
# prepare mapping capabilities (add basemaps and define mapping function)

# Add basemaps to folium
basemaps = {
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# prepare map

# create a folium map object, centered at RMBL
m = folium.Map(location=[38.9587,-106.9878], zoom_start=11)
# set visualization parameters
rgbVis = {'min': 0.0, 'max': 5000, 'bands': ['B4', 'B3', 'B2']}
vis = {'min':0, 'max':1, 'palette':'red, green'}

# add fullscreen button
plugins.Fullscreen().add_to(m)

In [ ]:
basemaps['Google Satellite'].add_to(m)

m.add_ee_layer(aspenMask2020, vis, 'Aspen Mask 2020')
m.add_ee_layer(aspen, {'color': 'yellow'}, 'aspen')
m.add_ee_layer(nonAspen, {'color': 'purple'}, 'non-aspen')

# add a layer control panel to the map
m.add_child(folium.LayerControl())

# display the map
display(m)
